# api文档
- 地理/逆地理编码
- 路径规划
- 行政区域查询
- 搜索POI
- IP定位
- 批量请求接口
- 静态地图坐标转换
- 天气查询
- 输入提示
- 交通态势
- 地理围栏
- 轨迹纠偏


## 地理/逆地理
- 地理编码/逆地理编码 API 是通过 HTTP/HTTPS 协议访问远程服务的接口，提供结构化地址与经纬度之间的相互转化的能力。

结构化地址的定义： 首先，地址肯定是一串字符，内含国家、省份、城市、区县、城镇、乡村、街道、门牌号码、屋邨、大厦等建筑物名称。按照由大区域名称到小区域名称组合在一起的字符。一个有效的地址应该是独一无二的。注意：针对大陆、港、澳地区的地理编码转换时可以将国家信息选择性的忽略，但省、市、城镇等级别的地址构成是不能忽略的。暂时不支持返回台湾省的详细地址信息。

In [1]:
import requests #用於API HTTP requests
# 使用python字典构造 parameters 请求参数
# API使用请求参数
key ='d9d5fa5d72ce3bcf4a2fee373f2e81af' # 高德地图API 获取Key
def geo(address:str,city:str)-> dict:
    """获取地理编码
    address可填入国家、省份、城市、区县、城镇、乡村、街道、门牌号码、屋邨、大厦等建筑物名称
    填入address对应city即可查询具体经纬度
    """
    parameters = {'key':key,
                  'city':city,
                  'citylimit': True,
                  'address':address
                   } 
    r = requests.get ("https://restapi.amap.com/v3/geocode/geo?parameters", params=parameters)
    data = r.json()['geocodes'][0]['location']
    return data

# 查询关键字(必填)   查询城市(可选)   仅返回指定城市数据(可选)



# requests 模塊


In [159]:
geo("广东省广州市从化区中山大学南方学院","广州")


'113.679287,23.632575'

In [160]:
geo("广东省广州市从化区温泉镇政府","广州")

'113.632401,23.616941'

## 路径规划
- 路径规划API是一套以HTTP形式提供的步行、公交、驾车查询及行驶距离计算接口，返回JSON 或 XML格式的查询数据，用于实现路径规划功能的开发。 由于道路/数据/算法的变更，很可能存在间隔一段时间后请求相同起终点的经纬度返回不同结果。

In [161]:
def walking(origin:str,destination:str)->dict:
    """路径规划
    输入起始点和终止点经纬度数据可获取路线规划
    可通过geo()函数查询具体地点的经纬度"""
    parameters = {
        'key':key,
        'origin':origin,
        'destination':destination,
        'output':'json'
    }
    r = requests.get("https://restapi.amap.com/v3/direction/walking?parameters",params = parameters)
    data = r.json()['route']['paths'][0]['steps']
    for i in range(0,len(data)):
        print(data[int(i)]['instruction'])
    
#     print(b[int(i)]['instruction'])
    
# a = walking(geo("广东省广州市从化区中山大学南方学院","广州"),geo("广东省广州市从化区温泉镇政府","广州"))
# b = a
# for i in range(0,len(b)):
#     print(b[int(i)]['instruction'])


In [162]:
walking(geo("广东省广州市从化区中山大学南方学院","广州"),geo("广东省广州市从化区温泉镇政府","广州"))

向南步行161米右转
向西步行99米左转
向南步行237米左转
向南步行498米向右前方行走
沿源富街向西南步行1568米直行
沿源湖街向西步行784米右转
沿龙泉路向西北步行1723米向左前方行走
沿龙泉路向西步行67米向左前方行走
沿温泉大道向西步行22米右转
步行13米左转
沿温泉大道向西南步行1425米右转
向西北步行96米右转
向北步行66米到达目的地


## 行政区域查询
- 行政区域查询是一类简单的HTTP接口，根据用户输入的搜索条件可以帮助用户快速的查找特定的行政区域信息。

In [163]:
def district(keywords:str):
    """行政区域查询
    输入省份即可得到对于行政区域"""
    parameters = {
        'key':key,
        'keywords':keywords,
        'output':'json'
    }
    r = requests.get ("https://restapi.amap.com/v3/config/district?parameters", params=parameters)
    data = r.json()['districts'][0]['districts']
    print("----"+keywords + "有以下几个行政区域----")
    for i in data:
        print(i['name'])
    return 
    
    

In [164]:
district('广东')

----广东有以下几个行政区域----
江门市
梅州市
广州市
河源市
东沙群岛
惠州市
清远市
肇庆市
佛山市
珠海市
云浮市
汕头市
茂名市
揭阳市
汕尾市
东莞市
深圳市
湛江市
阳江市
潮州市
韶关市
中山市


## 搜索POI
- 搜索服务API是一类简单的HTTP接口，提供多种查询POI信息的能力，其中包括关键字搜索、周边搜索、多边形搜索、ID查询四种筛选机制。

In [40]:
def poi(keywords:str,city:str):-> dict
    """搜索POI
    输入keywords关键词
    输入搜索的范围城市city
    即可得到相关信息的表格"""
    parameters = {
        'key':key,
        'city':city,
        'keywords':keywords,
        'output':'json',
        
    }
    r = requests.get ("https://restapi.amap.com/v3/place/text?parameters", params=parameters)
    data = r.json()
    pois = []
    pois.extend(data['pois'])
    import pandas as pd
    df_input = pd.DataFrame(pois)
    return df_input
#     if out == True:
#         df_input['keywords']=keywords
# #         select_fields = "keywords	id	name	type	typecode	location	tel	pname	pcode	cityname	citycode	adname	adcode".split("\t")
# #         df = df_input[select_fields]
# #         df = df.assign( 经 = [x.split(',')[0] for x in df['location']])
# #         df = df.assign( 纬 = [x.split(',')[1] for x in df['location']])
#         df.to_csv("keywords.csv", encoding="utf8", sep='\t') 
#     else:
        
    

In [42]:
data = poi("杨国福","广东")
data
# # pois = []
# # pois.extend(data['pois'])
# # import pandas as pd
# # df_input = pd.DataFrame(pois)
# # df_input

,address,adname,biz_ext,biz_type,childtype,cityname,distance,id,importance,location,name,parent,photos,pname,poiweight,shopid,shopinfo,tel,type,typecode
0,惠城镇葵和大道62号祚通农贸大楼下铺面第二间,惠来县,"{'rating': '3.5', 'cost': '26.00', 'meal_order...",diner,[],揭阳市,[],B0FFHO0PJ5,[],"116.300713,23.029194",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],0,13376522066,餐饮服务;中餐厅;中餐厅,050100
1,兴东路216号,兴宁市,"{'rating': '3.0', 'cost': [], 'meal_ordering':...",diner,[],梅州市,[],B0FFK6AO6S,[],"115.737061,24.139342",杨国福麻辣烫,[],[],广东省,[],[],0,13823829317,餐饮服务;中餐厅;中餐厅,050100
2,龙仙镇龙英路49-51号,翁源县,"{'rating': '3.0', 'cost': [], 'meal_ordering':...",diner,[],韶关市,[],B0FFICW4T3,[],"114.127759,24.360414",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],1,13538774273;15347533527,餐饮服务;中餐厅;中餐厅,050100
3,高平大道89号之一第四卡之二,中山市,"{'rating': '3.0', 'cost': [], 'meal_ordering':...",diner,[],中山市,[],B0FFI2K9H3,[],"113.466349,22.698194",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],0,13662534257,餐饮服务;餐饮相关场所;餐饮相关,050000
4,甲子镇海滨公路117号伯公对面,陆丰市,"{'rating': '3.5', 'cost': [], 'meal_ordering':...",diner,[],汕尾市,[],B0FFIKVWXP,[],"116.076832,22.871600",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],0,0660-8959577,餐饮服务;快餐厅;快餐厅,050300
5,南村镇里仁洞新兴大道12号,番禺区,"{'rating': '3.0', 'cost': '29.00', 'meal_order...",diner,[],广州市,[],B0FFF9279F,[],"113.344745,22.995019",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],0,020-84789414;15138682983,餐饮服务;中餐厅;特色/地方风味餐厅,050118
6,罗城柑园工业一路53号首层,罗定市,"{'rating': [], 'cost': [], 'meal_ordering': '0'}",diner,[],云浮市,[],B0FFL0N4VB,[],"111.578200,22.764173",杨国福麻辣烫,[],[{'url': 'http://aos-cdn-image.amap.com/sns/ug...,广东省,[],[],0,18029496469,餐饮服务;餐饮相关场所;餐饮相关,050000
7,大旺高新区政德大街1号万都广场首层4号商铺,四会市,"{'rating': [], 'cost': [], 'meal_ordering': '0'}",diner,[],肇庆市,[],B0FFKMU8W2,[],"112.809944,23.296791",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],1,13114517128,餐饮服务;快餐厅;快餐厅,050300
8,水闸路25座紫金广场对面家惠大厦旁边,四会市,"{'rating': '3.0', 'cost': [], 'meal_ordering':...",diner,[],肇庆市,[],B0FFJJ3J21,[],"112.698333,23.343140",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],0,15900140713,餐饮服务;中餐厅;中餐厅,050100
9,南门大街60号,惠阳区,"{'rating': [], 'cost': [], 'meal_ordering': '0'}",diner,[],惠州市,[],B0FFJ9L14K,[],"114.468653,22.793365",杨国福麻辣烫,[],[{'url': 'http://store.is.autonavi.com/showpic...,广东省,[],[],0,[],餐饮服务;中餐厅;中餐厅,050100


### IP定位
- IP定位是一个简单的HTTP接口，根据用户输入的IP地址，能够快速的帮用户定位IP的所在位置。

In [165]:
def ip(ip:str):
    """IP定位可以帮你定位ip的所在位置，输入对应ip即可"""
    parameters = {
        'key':key,
        'ip':ip,
        'output':'json'
    }
    r = requests.get ("https://restapi.amap.com/v3/ip?parameters", params=parameters)
    data = r.json()
    del data['status']
    del data['info']
    df = ["省份：%s"%(data['province']),"地区：%s"%(data['city']),"地理编码：%s"%(data['adcode']),"经纬度： %s"%(data['rectangle'])]
    print("ip:"+ip)
    for i in df:
        print(i)
    return    
    
   
   
    
    

In [166]:
ip("182.254.192.28")

ip:182.254.192.28
省份：广东省
地区：广州市
地理编码：440100
经纬度： 113.1017375,22.93212254;113.6770499,23.3809537


### 静态地图

- 静态地图服务通过返回一张地图图片响应HTTP请求，使用户能够将高德地图以图片形式嵌入自己的网页中。用户可以指定请求的地图位置、图片大小、以及在地图上添加覆盖物，如标签、标注、折线、多边形。

- 静态地图在使用的过程中，需要遵守高德开放平台[《自定义地图服务协议》](https://lbs.amap.com/home/customize-map-terms/)。

In [43]:
def map(location:str,zoom:str):
    """zoom：设置地图大小
       location：选择地图中心坐标
    """
    parameters = {
        'key':key,
        'location':location,
        'zoom':zoom,
        
    }
    r = requests.get ("https://restapi.amap.com/v3/staticmap?parameters", params=parameters)
    data = r.url
    return data

In [44]:
map(geo("广东省广州市从化区中山大学南方学院","广州"),18)

'https://restapi.amap.com/v3/staticmap?parameters&key=d9d5fa5d72ce3bcf4a2fee373f2e81af&location=113.679287%2C23.632575&zoom=18'

![0](https://restapi.amap.com/v3/staticmap?parameters&key=d9d5fa5d72ce3bcf4a2fee373f2e81af&location=113.679287%2C23.632575&zoom=18)

In [235]:
# import matplotlib.pyplot as plt # plt 用于显示图片
# import matplotlib.image as mpimg # mpimg 用于读取图片
# import numpy as np
 
# lena = mpimg.imread(BytesIO(requests.get('https://restapi.amap.com/v3/staticmap?parameters&key=d9d5fa5d72ce3bcf4a2fee373f2e81af&location=113.679287%2C23.632575&zoom=8').content)) # 读取和代码处于同一目录下的 lena.png
# # 此时 lena 就已经是一个 np.array 了，可以对它进行任意处理
# lena.shape #(512, 512, 3)
 
# plt.imshow(lena) # 显示图片
# plt.axis('off') # 不显示坐标轴
# plt.show()


### 坐标转换
- 坐标转换是一类简单的HTTP接口，能够将用户输入的非高德坐标（GPS坐标、mapbar坐标、baidu坐标）转换成高德坐标。

In [11]:
def assistant(locations:str,coordsys:str):
    """坐标转换
    locations填入非高德坐标
    coordsys可选值：gps,mapbar,baidu,autonavi（不进行转换）
    """
    parameters = {
        'key':key,
        'coordsys':coordsys,
        'locations':locations,
        'output':'json'
    }
    r = requests.get ("https://restapi.amap.com/v3/assistant/coordinate/convert?parameters", params=parameters)
    data = r.json()['locations']
    return data

In [12]:
assistant("116.481499,39.990475|116.481499,39.990375",'gps')

'116.487585177952,39.991754014757;116.487585177952,39.991653917101'

### 天气查询
- 天气查询是一个简单的HTTP接口，根据用户输入的adcode，查询目标区域当前/未来的天气情况。

In [52]:
def weather(city:str):
    """天气查询
    查询当地天气情况"""
    parameters = {
        'key':key,
        'city':city,
        'output':'json'
    }
    r = requests.get ("https://restapi.amap.com/v3/weather/weatherInfo?parameters", params=parameters)
    data = r.json()
    
    return data

In [53]:
weather('广州')

{'status': '1',
 'count': '1',
 'info': 'OK',
 'infocode': '10000',
 'lives': [{'province': '广东',
   'city': '广州市',
   'adcode': '440100',
   'weather': '晴',
   'temperature': '23',
   'winddirection': '东北',
   'windpower': '≤3',
   'humidity': '70',
   'reporttime': '2019-10-16 23:45:09'}]}

### 输入提示
- 输入提示是一类简单的HTTP接口，提供根据用户输入的关键词查询返回建议列表。

In [177]:
def inputtips(keywords:str):
    """填写关键词课查询"""
    parameters = {
        'key':key,
        'keywords':keywords,
        'output':'json'
    }
    r = requests.get ("https://restapi.amap.com/v3/assistant/inputtips?parameters", params=parameters)
    data = r.json()
    return data

In [178]:
inputtips("中山大学")

{'status': '1',
 'count': '10',
 'info': 'OK',
 'infocode': '10000',
 'tips': [{'id': 'B00141IHRZ',
   'name': '中山大学广州校区南校园',
   'district': '广东省广州市海珠区',
   'adcode': '440105',
   'location': '113.298379,23.096705',
   'address': '新港西路135号',
   'typecode': '141201',
   'city': []},
  {'id': 'B02F402P24',
   'name': '中山大学珠海校区',
   'district': '广东省珠海市香洲区',
   'adcode': '440402',
   'location': '113.586266,22.349455',
   'address': '唐淇路',
   'typecode': '141201',
   'city': []},
  {'id': 'B0FFK8L6A0',
   'name': '中山大学北校区',
   'district': '广东省广州市越秀区',
   'adcode': '440104',
   'location': '113.290008,23.128634',
   'address': '东风东路658号',
   'typecode': '141201',
   'city': []},
  {'id': 'B00140NUN1',
   'name': '中山大学附属第六医院',
   'district': '广东省广州市天河区',
   'adcode': '440106',
   'location': '113.364148,23.118217',
   'address': '员村二横路26号(本部区)',
   'typecode': '090101',
   'city': []},
  {'id': 'B03017WMXK',
   'name': '中山大学附属东华医院',
   'district': '广东省东莞市',
   'adcode': '441900',
   'locatio

### 交通态势
- 交通态势是一类简单的HTTP接口，提供根据用户输入的内容能够返回希望查询的交通态势情况。

- 目前支持城市：北京，上海，广州，深圳，宁波，武汉，重庆，成都，沈阳，南京，杭州，长春，常州，大连，东莞，福州，青岛，石家庄，天津，太原，西安，无锡，厦门，珠海，长沙，苏州，金华，佛山，济南，泉州，嘉兴，西宁，惠州，温州，中山，合肥，乌鲁木齐，台州，绍兴，昆明。

In [63]:
def rectangle(rectangle:str):
    """rectangle:填写查询的经纬度"""
    parameters = {
        'key':key,
        'rectangle':rectangle,
        'output':'json'
    }
    r = requests.get ("https://restapi.amap.com/v3/traffic/status/rectangle?parameters", params=parameters)
    data = r.json()
    return data

In [64]:
rectangle("116.351147,39.966309;116.357134,39.968727")

{'status': '1',
 'info': 'OK',
 'infocode': '10000',
 'trafficinfo': {'description': '京藏高速：北沙滩桥附近出京方向行驶缓慢。',
  'evaluation': {'expedite': '92.86%',
   'congested': '7.14%',
   'blocked': '0.00%',
   'unknown': '0.00%',
   'status': '2',
   'description': '轻度拥堵'}}}

## 简单的应用

### 盲人出行路线指引应用
#### 一、所调用的高德API：
- 路径规划；
- 交通态势；
- IP定位

#### 二、产品需求：
- 现如今，关于人们出行的导航类软件层出不穷，越来越多的APP或者小程序都在开发类似的产品，但是专门做盲人出现路线的指导产品却少之又少。针对这个用户痛点，结合高德地图有关的API，打造一个专属盲人出现的路线指引，满足一部分用户的需求。

#### 三、产品描述：
- 盲人出行路线指引是以语音识别功能为基础的。用户根据语音提示输入要到达的目的地。应用再根据IP定位功能，实行路径规划和交通态势的分析，为用户选择最方便快捷、障碍最少或者交通状况最优的路线。帮助用户解决出行困难的问题。

#### 四、产品结构